<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Interacting with SQLite from Python


---

<a name="python-sqlite"></a>

---

<a id='sqlite3'></a>
### The SQLite3 Package

The command line utility can be useful for basic SQL tasks, but, because we're using Python, it is often easier to access SQLite directly through Python code. We can use the Python [SQLite3](https://docs.python.org/2.7/library/sqlite3.html) package for just this purpose.

**Below, we open a connection to an SQLite database file using Python. Like before, if the file does not already exist, it will be created automatically.**

In [7]:
import sqlite3
sqlite_db = 'test_db.sqlite' # create a database
conn = sqlite3.connect(sqlite_db) # connected database
c = conn.cursor() # for execute command line 

_Note: The `.cursor()` method is necessary because, in many DB structures, a cursor is needed to iterate over the rows of data. The cursor also functions as a means of fetching information when using SQLite3. _

**The syntax to create a table is similar to the console, only now we use the `.execute()` method of the cursor object (`c`) we just created:**

In [8]:
c.execute('CREATE TABLE houses (field1 INTEGER PRIMARY KEY, sqft INTEGER, bdrms INTEGER, age INTEGER, price INTEGER);')

# Save (commit) the changes.
conn.commit()

OperationalError: table houses already exists

With the database saved, the table should now be viewable among the existing tables.

<a id='sqlite3-adding'></a>

### SQLite3: Adding Data

Because we're back in Python, we can now use regular programming techniques in conjunction with the SQLite connection.  In particular, the cursor's `.execute()` method supports value substitution using the `?` character, which makes adding multiple records a bit easier.

See the [docs](https://docs.python.org/2.7/library/sqlite3.html) for more details.

In [9]:
last_sale = (None, 4000, 5, 22, 619000)
c.execute('INSERT INTO houses VALUES (?,?,?,?,?)', last_sale)

# Remember to commit the changes.
conn.commit()

Notice that, in this syntax, we use Python's `None` value, rather than `NULL`, to trigger SQLite and auto-increment the Primary Key. 

There is a related cursor method, `.executemany()`, which takes an array of tuples and loops through them, substituting one tuple at a time.

```python
recent_sales = [
  (None, 2390, 4, 34, 319000),
  (None, 1870, 3, 14, 289000),
  (None, 1505, 3, 90, 269000),
]
```

```python
c.executemany('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', recent_sales)
```

```python
conn.commit()
```
<a id='adding-csv'></a>

### Adding Data From a `.csv` File

Next, let's load our `housing.csv` data into an array and then `INSERT` those records into the database. In this example, we'll use the `numpy` `.genfromtxt()` function to read the file and parse its contents. 

In [11]:
from numpy import genfromtxt

# import into nparray of ints, then convert to list of lists
data = (genfromtxt('data/housing-data.csv', dtype='i8', 
                    delimiter=',', skip_header=1)).tolist()

# append a None value to beginning of each sub-list
for d in data:
    d.insert(0, None)

In [14]:
data[0:3]

[[None, 2104, 3, 70, 399900],
 [None, 1600, 3, 28, 329900],
 [None, 2400, 3, 44, 369000]]

In [23]:
# Loop through data, running an INSERT on each record (i.e., sublist).
for d in data:
    c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d)

conn.commit()

> **Remember:** All elements in a `numpy` array must be the same data type, so, if we want to "add a `None`" to each row, we would need to work around this. Lists can contain mixed types, so this is one approach.

Still, in this case, the value we're adding is the same for all records, so we can simply use a `None` in the INSERT statement.

**How do you delete data?**

In [27]:
# Similar syntax as before.
results = c.execute("SELECT * FROM houses")


In [28]:
# Here, results is a cursor object — use `.fetchall()` to extract a list.
len(results.fetchall())

48

In [29]:
# Similar syntax as before.
results = c.execute("SELECT * FROM houses WHERE bdrms = 4")

# Here, results is a cursor object — use `.fetchall()` to extract a list.
results.fetchall()

[(6, 3000, 4, 75, 539900),
 (7, 1985, 4, 61, 299900),
 (12, 1940, 4, 7, 239999),
 (17, 2300, 4, 77, 449900),
 (20, 2609, 4, 5, 499998),
 (21, 3031, 4, 21, 599000),
 (25, 1962, 4, 53, 259900),
 (34, 2040, 4, 75, 314900),
 (36, 1811, 4, 24, 285900),
 (39, 2132, 4, 28, 345000),
 (40, 4215, 4, 66, 549000),
 (41, 2162, 4, 43, 287000),
 (44, 2567, 4, 57, 314000),
 (47, 1852, 4, 64, 299900)]

In [45]:
#data
len([d for d in data if d[3]>50])

18

In [47]:
# Similar syntax as before.
results = c.execute("Delete FROM houses where age>50")
results.fetchall()

[]

In [48]:
results = c.execute("SELECT * FROM houses")

len(results.fetchall())

30

<a id='pandas-connector'></a>

## `pandas` Connector

---

While databases provide many analytical capabilities, it's often useful to pull data back into Python for more flexible programming. Large, fixed operations would be more efficient in a database, but `pandas` allows for interactive processing.

For example, imagine you want to aggregate nightly logins or sales to present a report or dashboard. Most likely, you wouldn't be changing or operating on a large data set. You could perform this aggregation efficiently within a database.

However, if we want to investigate login or sales data further and ask more interactive questions, using Python would be more practical.

In [52]:
import pandas as pd

`Pandas` can connect to most relational databases. In this demonstration, we'll create and connect to an SQLite database.

SQLite creates portable SQL databases saved in a single file. These databases are stored efficiently and allow for  fast querying, making them ideal for small databases or ones that need to be moved across machines.

<a id='pandas-writing'></a>

### Writing Data Into a Database

Data in `pandas` can be loaded into a relational database. For the most part, `pandas` can use column information to infer the schema for the table it creates. In the next demo, we will use the housing data set.

In [56]:
data = pd.read_csv('data/housing-data.csv')
data.head()

,sqft,bdrms,age,price
0,2104,3,70,399900
1,1600,3,28,329900
2,2400,3,44,369000
3,1416,2,49,232000
4,3000,4,75,539900


In [60]:
data.describe().T # transport data becuase it's matrix 


,count,mean,std,min,25%,50%,75%,max
sqft,47.0,2000.680851,794.702354,852.0,1432.0,1888.0,2269.0,4478.0
bdrms,47.0,3.170213,0.760982,1.0,3.0,3.0,4.0,5.0
age,47.0,42.744681,22.873440,5.0,24.5,44.0,61.5,79.0
price,47.0,340412.659574,125039.899586,169900.0,249900.0,299900.0,384450.0,699900.0


Data is moved to the database through the `to_sql` command, similar to the `to_csv` command.

`to_sql` takes the following arguments:
- `name`, the table name to create.
- `con`, a connection to a database.
- `index`, whether or not to input the index column.
- `schema`, if we want to write a custom schema for the new table.
- `if_exists`, what to do if the table already exists. We can overwrite it, add to it, or fail.
    
> ### BTW, what was with the `conn` object?  
> We defined it in a previous example:
> ```python
import sqlite3
sqlite_db = './datasets/test_db.sqlite'
conn = sqlite3.connect(sqlite_db) 
c = conn.cursor()
```

In [61]:
data.to_sql('houses_pandas',             # Name of the table.
            con=conn,                    # The handle to the file that is set up.
            if_exists='replace',         # Overwrite, append, or fail.
            index=False)                 # Add index as column.

<a id='pandas-reading'></a>

### Reading Data from a Database
_Result as a DataFrame_

If we already have data in our database, we can use `pandas` to query it. Querying is done through the `read_sql` command in the `sql` module.

In [62]:
df = pd.read_sql('SELECT * FROM houses_pandas LIMIT 10', con=conn)
df

,sqft,bdrms,age,price
0,2104,3,70,399900
1,1600,3,28,329900
2,2400,3,44,369000
3,1416,2,49,232000
4,3000,4,75,539900
5,1985,4,61,299900
6,1534,3,12,314900
7,1427,3,57,198999
8,1380,3,14,212000
9,1494,3,15,242500


In [74]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
sqft,10.0,1834.0,535.667600,1380.0,1443.75,1567.0,2074.25,3000.0
bdrms,10.0,3.1,0.567646,2.0,3.00,3.0,3.00,4.0
age,10.0,42.5,23.847898,12.0,18.25,46.5,60.00,75.0
price,10.0,313899.9,104004.503001,198999.0,234625.00,307400.0,359225.00,539900.0


<a name="sql-syntax"></a>

## SQL Syntax

---

### SQL Operators

> Note: Each of the following can be demonstrated in `pandas` using the data we've set up above. A demo and check are included for each, but we can try out a few before we begin.

#### SELECT

Every query should start with `SELECT`. `SELECT` is followed by the names of the columns in the output.

`SELECT` is always paired with `FROM`, which identifies the table from which we're retrieving the data.

```SQL
SELECT
<columns>
FROM
<table>
```

`SELECT *` returns *all* of the columns.

Housing data example:
```SQL
SELECT
*
FROM houses_pandas;
```

**Check:** What do `sqft`, `bdrms`, and `price` refer to? What will they return?

```SQL
SELECT
sqft, bdrms, price
FROM houses_pandas;
```

#### WHERE

`WHERE`, which follows the `FROM` clause, is used to filter tables using specific criteria.

```SQL
SELECT
<columns>
FROM
<table>
WHERE
<condition>
```
Example:
```SQL
SELECT
sqft, bdrms, age, price
FROM houses_pandas
WHERE bdrms = 2 and price < 250000;
```

The condition is a filter that's applied to the rows. The rows that match the condition will be included in the output.

**Check:** Write a query that returns the `sqft`, `bdrms`, and `age` for houses older than 60.

<a name="aggregations"></a>

### Aggregations

Aggregations (or aggregate functions) are functions in which the values of multiple rows are grouped together as an input on certain criteria to form a single value of more significant meaning or measurement. Examples are sets, bags, or lists.

Aggregate functions include:

- Average (i.e., arithmetic mean)
- Count
- Maximum
- Minimum
- Median
- Mode
- Sum

In SQL, they are performed in a `SELECT` statement like the following:

```SQL
SELECT COUNT(price)
FROM houses_pandas;
```

```SQL
SELECT AVG(sqft), MIN(price), MAX(price)
FROM houses_pandas
WHERE bdrms = 2;
```

In [77]:
sql_query = (
    'SELECT AVG(sqft), MIN(price), MAX(price) '
    'FROM houses_pandas '
    'WHERE bdrms = 2') 


# We can combine SQL queries with `pandas` functions for aggregate info, too.
pd.read_sql(sql_query, con=conn)

,AVG(sqft),MIN(price),MAX(price)
0,1496.5,179900,368500


In [78]:
sql_query = "SELECT * FROM houses_pandas WHERE bdrms=2" #Single-line query

# We can combine SQL queries with `pandas` functions for aggregate info, too.
pd.read_sql(sql_query, con=conn).describe().T

,count,mean,std,min,25%,50%,75%,max
sqft,6.0,1496.500000,387.723484,852.0,1344.00,1540.0,1795.25,1888.0
bdrms,6.0,2.000000,0.000000,2.0,2.00,2.0,2.00,2.0
age,6.0,56.666667,16.243973,40.0,42.25,55.5,68.00,79.0
price,6.0,280866.666667,72192.483450,179900.0,237750.00,277450.0,337400.00,368500.0


### Group by

We can aggregate only within groups using the `GROUP BY` argument. 

Any `WHERE` statements have to come before `GROUP BY` statements.

In [99]:
sql_query = (
    'SELECT bdrms, AVG(sqft), MIN(price), MAX(price) '
    'FROM houses_pandas '
    'WHERE age<50 '
    'GROUP BY bdrms') 


# We can combine SQL queries with `pandas` functions for aggregate info, too.
pd.read_sql(sql_query, con=conn, index_col='bdrms')


,AVG(sqft),MIN(price),MAX(price)
bdrms,,,
1,1000.000000,169900,169900
2,1639.666667,232000,368500
3,1944.705882,212000,573900
4,2280.833333,239999,599000
5,4478.000000,699900,699900


That is equivalent to selecting first and then using the pandas group by.

In [100]:
sql_query = (
    'SELECT bdrms, sqft, price '
    'FROM houses_pandas '
    'WHERE age < 50') 


# We can combine SQL queries with `pandas` functions for aggregate info, too.
groups = pd.read_sql(sql_query, con=conn).groupby('bdrms')

In [101]:
df.describe().T
groups.sqft.mean()

bdrms
1    1000.000000
2    1639.666667
3    1944.705882
4    2280.833333
5    4478.000000
Name: sqft, dtype: float64

In [102]:
groups.price.min()

bdrms
1    169900
2    232000
3    212000
4    239999
5    699900
Name: price, dtype: int64

In [103]:
groups.price.max()


bdrms
1    169900
2    368500
3    573900
4    599000
5    699900
Name: price, dtype: int64

<a name="ind-practice"></a>

## Independent Practice: Querying a Database

---

Practice querying the SQLite database we've created using any of the methods you've learned so far:

- Console connection
- Python's `sqlite3` package
- `pandas`

### What is the average price per room for two-bedroom apartments?

### How old is the oldest three-bedroom apartment?

### How old is the youngest apartment?

### What is the average age for the whole data set?

### What is the average age for each bedroom size?
_Tip: This will most likely require `pandas`/SQL integration._

**If you finish, try completing the first sections of [SQL Zoo](http://www.sqlzoo.net).**

## SQLite Browser

There are two options you might want to explore for browsing SQLite3 databases:

- [SQLite Browser](http://sqlitebrowser.org/), a free, cross-platform solution.
- [SQLite Manager](https://addons.mozilla.org/en-US/firefox/addon/sqlite-manager/?src), a Firefox add-on for viewing SQLite database files via a simple GUI.


You can install SQLite Browser with

```bash
brew cask install db-browser-for-sqlite
```

If you don't have brew installed, execute first the following command:

```
/usr/bin/ruby -e "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/master/install)"
```

Now that we have some data, take a look at the database using the **SQLite Browser**.

1. Open the app.
2. Open SQLite3 file "Open Database" from the file menu. 
   - _This file will most likely be in the directory you were in when we started using `sqlite3` commands._
3. After connecting to the database, click on "Browse Data" in the tabbed UI.

You should be able to view something like below. 

![SQLite Manager](https://snag.gy/xc8W5M.jpg)

<a name="remote-database"></a>
## Connecting to a Remote Database

---


<a id='postgresql'></a>
### PostgreSQL

**[PostgreSQL](http://www.postgresql.org/)** is a powerful SQL-based relational database.

#### PostgreSQL Syntax

GA provides a PostgreSQL database instance at the following address:

You can connect to it using:

> `psql -h dsi.c20gkj5cvu3l.us-east-1.rds.amazonaws.com -p 5432 -U dsi_student titanic`  
    Password: gastudents

PosgreSQL accepts the same syntax as SQLite, with exception of a few system commands. Here are the most common:

- `\q`: Quit/exit.
- `\c __database__`: Connect to a database.
- `\d __table__`: Show table definition, including triggers.
- `\dt *.*`: List tables from all schemas (if `*.*` is omitted, this will only show SEARCH_PATH ones).
- `\l`: List databases.
- `\dn`: List schemas.
- `\df`: List functions.
- `\dv`: List views.
- `\dt`: List tables.
- `\df+ __function`: Show function SQL code.
- `\x`: Pretty-format query results instead of displaying not-so-useful ASCII tables.


Suggestions:

- Most `\d` commands support additional parameters of `__schema__.name__` and accept wildcards like `*.*`.

<a id='comparison-commands'></a>
## Comparison of SQL Commands

---


|Task|MySQL|PostgreSQL|SQLite|
|---|---|---|---|
|Connect to a database|mysql &lt;dbname&gt;| psql &lt;dbname&gt;|sqlite3 &lt;filename&gt;|
|Client help|help contents|\?|.help|
|SQL help|help contents|\h|n/a|
|List databases|SHOW DATABASES;|\l|.databases|
|Change database|USE &lt;dbname&gt;|\c <dbname&gt;|n/a|
|List tables|SHOW TABLES;|\dt|.tables|
|Show table|info DESCRIBE &lt;tablename&gt;|\d &lt;tablename&gt;|.schema &lt;tablename&gt;|
|Load data|LOAD DATA INFILE &lt;file&gt; |\i &lt;file&gt;|.import &lt;file&gt; &lt;table&gt;|
|Export data|SELECT ... INTO OUTFILE &lt;file&gt;|\o &lt;file&gt;|.dump &lt;table&gt;|
|Exit the client|quit (or exit)| \q|.exit|

<a name="intermission"></a>

## Conclusion

---

We have seen how to connect to a local SQLite database and a remote postgreSQL database.

**Check:** What SQL operations have we learned so far?

**Check:** What commands have we learned for SQLite and PostgreSQL?


<a id='resources'></a>

## Additional Resources 

---

- [SQLite3's website](http://www.sqlite.org)  
- [SQLite — Python tutorial](http://sebastianraschka.com/Articles/2014_sqlite_in_python_tutorial.html)  
- [SQL Zoo](http://www.sqlzoo.net)